# Exercise 15: Power Analyses

This  assignment is designed to give you practice with Monte Carlo methods to conduct power analyses via simulation. You won't need to load in any data for this homework. We will, however, be using parts of the homework from last week.

---
## 1. Simulating data (1 points)


Pull your `simulate_data()` function from your last homework and add it below. 

As a reminder, this function simulates the relationship between age, word reading experience, and reading comprehension skill. 

`c` is reading comprehension, and `x` is word reading experience.

In [1]:
sample_size = 100 # How many children in data set? 
age_lo = 80     # minimum age, in months
age_hi = 200    # maximum age, in months
beta_xa = 0.5   # amount by which experience changes for increase of one month in age
beta_x0 = -5    # amount of experience when age = 0 (not interpretable, since minimum age for this data is 80 months)
sd_x = 50       # standard dev of gaussian noise term, epsilon_x
beta_ca = 0.8   # amount that comprehension score improves for every increase of one unit in age
beta_cx = 3     # amount that comprehension score improves for every increase of one unit in reading experience
beta_c0 = 10    # comprehension score when reading experience is 0. 
sd_c = 85      # standard dev of gaussian noise term, epsilon_c

simulate_data <- function(sample_size, age_lo, age_hi, beta_xa, 
                          beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
      # WRITE YOUR CODE HERE
      age <- runif(n = sample_size,min = age_lo,max = age_hi)

      x <- beta_xa*age + beta_x0 + rnorm(n = sample_size,sd = sd_x)
        
      c <- beta_ca*age + beta_cx*x + beta_c0 + rnorm(n = sample_size,sd = sd_c)

      return(data.frame(Age = age, Experience = x, Comprehension = c))
}

dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
head(dat)

,Age,Experience,Comprehension
,<dbl>,<dbl>,<dbl>
1,108.7343,53.82182,153.35079
2,184.3343,24.87975,73.53553
3,107.9788,53.76689,244.75646
4,189.9121,96.90713,401.60573
5,184.3163,-32.26344,181.29409
6,139.5223,40.68676,400.07175


---
## 2. `run_analysis()` function (2 pts)

Last week, we looked at whether word reading experience(`x`) mediated the relation between `age` and reading comprehension (`c`).

Now we're going to use our `simulate_data()` function to conduct a power analysis. The goal is to determine how many participants we would need in order to detect both the mediated and the direct effects in this data. 

*Note: We're going to pretend for the sake of simplicity that we don't have any control over the ages of the children we get (so ages are generated using `runif(sample_size, age_lo, age_hi)`, although of course this would be an unusual situation in reality.*

First, write a function, `run_analysis()`, that takes in simulated data, runs **your mediation from last week**, and returns a vector containing the ACME and ADE estimates and p-values (these are the `d0`, `d0.p`, `z0`, and `z0.p` features of the mediated model object, e.g., `fitMed$d0.p`). Print this function's output for the data we simulated previously. 

In [2]:
# WRITE YOUR CODE HERE
install.packages("mediation")
library(mediation)

run_analysis <- function(dat) {
  
  m1 <- lm(Experience ~ Age, dat)
  m2 <- lm(Comprehension ~ Experience + Age, dat)

  model <- mediate(m1,m2, treat = "Age", mediator = "Experience") 

  d0 <- model$d0 
  d0.p <- model$d0.p 

  z0 <- model$z0 
  z0.p <- model$z0.p 

  return(c(d0, d0.p, z0, z0.p))
}
   
run_analysis(dat)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘checkmate’, ‘htmlwidgets’, ‘zoo’, ‘gridExtra’, ‘htmlTable’, ‘viridis’, ‘Formula’, ‘minqa’, ‘nloptr’, ‘Rcpp’, ‘RcppEigen’, ‘mvtnorm’, ‘sandwich’, ‘lpSolve’, ‘Hmisc’, ‘lme4’


Loading required package: MASS

Loading required package: Matrix

Loading required package: mvtnorm

Loading required package: sandwich

mediation: Causal Mediation Analysis
Version: 4.5.0




[1] 1.576304 0.000000 1.023585 0.000000

---
## 3. `repeat_analysis()` function (3 pts)

Next fill in the function `repeat_analysis()` below so that it simulates and analyzes data `num_simulations` times. Store the outputs from each simulation in the `simouts` matrix. Calculate and return the coverage across all the simulations run for both ACME and ADE.

In [3]:
repeat_analysis <- function(num_simulations, alpha, sample_size, age_lo, age_hi, 
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
    # Initialize simouts matrix for storing each output from run_analysis()
    simouts <- matrix(rep(NA, num_simulations*4), nrow=num_simulations, ncol=4)
    
    # Start simulating
    for (i in 1:num_simulations) {
      # WRITE YOUR CODE HERE
      dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x,
                           beta_ca, beta_cx, beta_c0, sd_c)

     simouts[i,] <- run_analysis(dat)
    }
    
    ACME_cov =  mean(simouts[,2]<=alpha)
    ADE_cov =  mean(simouts[,4]<=alpha)

    return(list(ACME_cov = ACME_cov, ADE_cov = ADE_cov))
}

Now run the `repeat_analysis()` function using the same parameter settings as above, for 10 simulations, with an alpha criterion of 0.01. 

In [4]:
# WRITE YOUR CODE HERE
repeat_analysis(num_simulations = 10, alpha = 0.01, sample_size, age_lo, age_hi, 
                beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)


$ACME_cov
[1] 0.8

$ADE_cov
[1] 1

---
## 4. Testing different sample sizes (2 pts)

Finally, do the same thing (10 simulations, alpha criterion of 0.01) but for 5 different sample sizes: 50, 75, 100, 125, 150. You can do this using `map` (as in the tutorial), or a simple `for` loop, or by calculating each individually. Up to you! This should take around 3 minutes to run. 

In [6]:
# WRITE YOUR CODE HERE
require(tidyverse)

n <- c(50, 75, 100, 125, 150)
num_sample_size <- length(n)
df <- matrix(NA, ncol = 3, nrow = 5)
colnames(df) <- c("num_sim", "ACME_cov", "ADE_cov")


for (i in 1:num_sample_size) {
  sample_size = n[i]
  df[i,"num_sim"] <- sample_size
  
  out <- repeat_analysis(num_simulations = 10, alpha = 0.01,
                        sample_size = sample_size, age_lo,
                        age_hi, beta_xa, beta_x0, sd_x, beta_ca,
                        beta_cx, beta_c0, sd_c)
  
  df[i, "ACME_cov"] <- out$ACME_cov
  df[i, "ADE_cov"] <- out$ADE_cov
    
}

Print your results. 

In [8]:
# WRITE YOUR CODE HERE
head(df)

num_sim,ACME_cov,ADE_cov
50,0.3,0.2
75,0.6,0.6
100,0.7,0.7
125,0.8,0.7
150,1.0,0.8


## 5. Reflection (2 pts)

If this were a real power analysis, we'd want to run more simulations per sample size (to get a more precise estimate of power) and we may also want to test out some other values of the parameters we used to simulate our data. However, what would you conclude just based on the results above? 

> *Write your response here.* 
> 
Coverage increases as sample size increases. When we get to sample size of 150, we are 80% likely to correctly reject the null.  

**Given** how we generated the data, why was the direct effect harder to detect than the mediated effect?
> *Write your response here.* 
The way we generated data, we used rnorm which scales our data which isn't ideal and made it harder to detect the direct effect. 

**DUE:** 5pm EST, April 5, 2023

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> *Someone's Name*